In [1]:
import pandas as pd
import itertools
import numpy as np
import statsmodels.imputation.mice as mice
import statsmodels.api as sm

In [2]:
data4 = pd.read_csv('~/Desktop/data4.csv')

In [3]:
resistances = data4[['DSTAm','DSTCm','DSTKm','DSTS','DSTOfx','DSTH','DSTR']].dropna(axis = 1).applymap(lambda x: 1 if x == 'R' else 0 if x == 'S' else np.nan)

In [4]:
used_drugs = data4[['Am','Cm','Km','S','Ofx','H','R']].dropna(axis = 1).applymap(lambda x: 1 if x == 'Yes' else 0 if x == 'No' else np.nan)

In [5]:
age = data4['Age'].map(lambda x: 1 if x <= 25 else 2 if (x > 25)&(x <= 45) else 3 if x > 45 else np.nan)
age_d = pd.get_dummies(age, 'age', drop_first=True)
sex = data4['Sex'].map(lambda x: 1 if x[0] == 'F' else 0 if x == 'M' else np.nan) 
smoking = data4['Smoking'].map(lambda x: 1 if x == 'Yes' else 0 if x == 'No' else np.nan)
#art = data4['ART'].map(lambda x: 1 if x == 'Yes' else 0 if x == 'No' else np.nan)
#hiv = data4['HIV'].map(lambda x: 1 if x == 'Pos' else 0 if x == 'Neg' else np.nan)
#pasttx = data4['PastTx'].map(lambda x: 1 if x == 'Yes' else 0 if x == 'No' else np.nan)
#pasttype = data4['PastType'].map(lambda x: 1 if x == 'FLD' else 2 if x == 'SLD' else np.nan)
#site = data4['Site'].map(lambda x: 1 if x == 'Pulm' else 2 if x == 'Extrapulm' else 3 if x=='Both' else np.nan)
site_d = pd.get_dummies(data4['Site'], 'site', drop_first=True)

aintno = data4['AIntNo']
acontno = data4['AContNo']
atotaldur = data4['ATotalDur']

hiv_art = data4[['HIV', 'ART']].apply(lambda x: 'ART' if x[1] == 'Yes' else 'HIV' if (x[1] != 'Yes' and x[0] == 'Pos') else 'Neg' if x[0] == 'Neg' else np.nan, axis = 1).rename('Hiv_art')
hiv_art_d = pd.get_dummies(hiv_art, 'hiv_art', drop_first=True)

past = data4[['PastTx', 'PastType']].apply(lambda x: 'SLD' if x[1] == 'SLD' else 'FLD' if x[1] == 'FLD' else 'Some' if x[0] == 'Yes' else 'No' if x[0] == 'No' else np.nan, axis = 1).rename('Past')
past_d = pd.get_dummies(past, 'past', drop_first=True)

covariates = pd.concat([age_d, sex, smoking, hiv_art_d, past_d, site_d, aintno, acontno, atotaldur], axis=1)

In [6]:
outcomes = data4[['Outcome', 'Relapse']].apply(lambda x: 1 if (x[0] == 'Death' or x[0] == 'Lost' or x[0] == 'Fail' or x[0] == 'Transfer' or x[1] == 'Yes') else 0, axis = 1).rename('Outcome')

In [7]:
full = pd.concat([covariates, used_drugs, resistances, outcomes], axis=1)
full

,age_2,age_3,Sex,Smoking,hiv_art_HIV,hiv_art_Neg,past_No,past_SLD,past_Some,site_Extrapulm,...,H,R,DSTAm,DSTCm,DSTKm,DSTS,DSTOfx,DSTH,DSTR,Outcome
0,0,1,0.0,NaN,0,0,0,0,1,0,...,0,0,0.0,0.0,1.0,1.0,0.0,1,1,1
1,0,1,1.0,NaN,0,1,0,0,1,0,...,0,0,NaN,NaN,NaN,NaN,NaN,1,1,0
2,1,0,1.0,NaN,0,1,1,0,0,0,...,0,0,0.0,NaN,NaN,0.0,0.0,1,1,0
3,1,0,0.0,NaN,0,1,0,0,1,0,...,0,0,0.0,NaN,NaN,1.0,0.0,1,1,0
4,1,0,1.0,NaN,0,1,1,0,0,0,...,0,0,0.0,NaN,NaN,1.0,0.0,1,1,0
5,1,0,1.0,NaN,0,1,1,0,0,0,...,0,0,0.0,NaN,NaN,1.0,1.0,1,1,0
6,1,0,0.0,NaN,0,1,1,0,0,0,...,0,0,0.0,NaN,NaN,0.0,0.0,1,1,1
7,1,0,0.0,NaN,0,1,1,0,0,0,...,0,0,1.0,1.0,NaN,1.0,1.0,1,1,0
8,1,0,1.0,NaN,0,1,1,0,0,0,...,0,0,0.0,NaN,NaN,0.0,0.0,1,1,0
9,0,0,1.0,NaN,0,1,1,0,0,0,...,0,0,0.0,NaN,NaN,0.0,0.0,1,1,0


In [8]:
imp = mice.MICEData(full)
imp.update_all(10)
imputed = imp.data

/Users/ltemereva/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
imputed

,age_2,age_3,Sex,Smoking,hiv_art_HIV,hiv_art_Neg,past_No,past_SLD,past_Some,site_Extrapulm,...,H,R,DSTAm,DSTCm,DSTKm,DSTS,DSTOfx,DSTH,DSTR,Outcome
0,0,1,0.0,1.0,0,0,0,0,1,0,...,0,0,0.0,0.0,1.0,1.0,0.0,1,1,1
1,0,1,1.0,0.0,0,1,0,0,1,0,...,0,0,0.0,0.0,0.0,0.0,0.0,1,1,0
2,1,0,1.0,0.0,0,1,1,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,1,1,0
3,1,0,0.0,0.0,0,1,0,0,1,0,...,0,0,0.0,0.0,0.0,1.0,0.0,1,1,0
4,1,0,1.0,0.0,0,1,1,0,0,0,...,0,0,0.0,0.0,0.0,1.0,0.0,1,1,0
5,1,0,1.0,0.0,0,1,1,0,0,0,...,0,0,0.0,0.0,0.0,1.0,1.0,1,1,0
6,1,0,0.0,1.0,0,1,1,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,1,1,1
7,1,0,0.0,1.0,0,1,1,0,0,0,...,0,0,1.0,1.0,1.0,1.0,1.0,1,1,0
8,1,0,1.0,0.0,0,1,1,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,1,1,0
9,0,0,1.0,0.0,0,1,1,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,1,1,0


In [30]:
res_injectables = (imputed['DSTAm'] == 1) | (imputed['DSTCm'] == 1) | (imputed['DSTKm'] == 1) | (imputed['DSTS'] == 1)
#res_fq = (imputed['DSTCfx'] == 'R') | (imputed['DSTLfx'] == 'R') | (imputed['DSTMfx'] == 'R') | (imputed['DSTOfx'] == 'R')

#Conditions for 2 fq from 4
#res_fq = list(itertools.combinations(['DSTLfx == R', 'DSTCfx == R', 'DSTMfx == R', 'DSTOfx == R'], 2))
#ands = list(map(lambda x: " & ".join(x), comb))
#cond = " | ".join(ands)

res_fq = (imputed['DSTOfx'] == 1)
res_first_line = (imputed['DSTH'] == 1) & (imputed['DSTR'] == 1)

res_Ofx = (imputed['DSTOfx'] == 1) # only Ofx can be analysed in this dataset
#res_Cfx = (imputed['DSTCfx'] == 'R') & (imputed['DSTLfx'] != 'R') & (imputed['DSTMfx'] != 'R') & (imputed['DSTOfx'] != 'R') 
#res_Mfx = (imputed['DSTMfx'] == 'R') & (imputed['DSTLfx'] != 'R') & (imputed['DSTCfx'] != 'R') & (imputed['DSTOfx'] != 'R')
#res_Lfx = (imputed['DSTLfx'] == 'R') & (imputed['DSTCfx'] != 'R') & (imputed['DSTMfx'] != 'R') & (imputed['DSTOfx'] != 'R')
#res_Ofx = (imputed['DSTOfx'] == 'R') & (imputed['DSTLfx'] != 'R') & (imputed['DSTMfx'] != 'R') & (imputed['DSTCfx'] != 'R')
res_Am = (imputed['DSTAm'] == 1) & (imputed['DSTKm'] != 1) & (imputed['DSTCm'] != 1) & (imputed['DSTS'] != 1)
res_Cm = (imputed['DSTCm'] == 1) & (imputed['DSTKm'] != 1) & (imputed['DSTAm'] != 1) & (imputed['DSTS'] != 1)
res_Km = (imputed['DSTKm'] == 1) & (imputed['DSTAm'] != 1) & (imputed['DSTCm'] != 1) & (imputed['DSTS'] != 1)
res_S = (imputed['DSTS'] == 1) & (imputed['DSTKm'] != 1) & (imputed['DSTCm'] != 1) & (imputed['DSTAm'] != 1)

res_first_line_only = res_first_line & ~res_injectables & ~res_fq
res_one_fq = res_Ofx & res_first_line & ~res_injectables
#res_one_fq = (res_Cfx | res_Mfx | res_Lfx | res_Ofx) & res_first_line & ~res_injectables
res_one_injectables = (res_Am | res_Cm | res_Km | res_S) & res_first_line & ~res_fq
res_injectables_only = res_first_line & res_injectables & ~res_fq
res_fq_only = res_first_line & res_fq & ~res_injectables
res_fq_injectables = res_injectables & res_fq & res_first_line

In [31]:
res_first_line_m = res_first_line.map({True: 1, False: -1}).rename('FirstLine')
res_first_line_only_m = res_first_line_only.map({True: 1, False: -1}).rename('FirstLineOnly')
res_one_injectables_m = res_one_injectables.map({True: 1, False: -1}).rename('OneInj')
res_one_fq_m = res_one_fq.map({True: 1, False: -1}).rename('OneFq')
res_injectables_only_m = res_injectables_only.map({True: 1, False: -1}).rename('InjOnly')
res_fq_only_m = res_fq_only.map({True: 1, False: -1}).rename('FqOnly')
res_fq_injectables_m = res_fq_injectables.map({True: 1, False: -1}).rename('FqInjectables')
variables = pd.concat([res_first_line_m, res_first_line_only_m, res_one_injectables_m, res_one_fq_m, res_injectables_only_m, res_fq_only_m, res_fq_injectables_m], axis=1)

In [32]:
model_data = pd.concat([covariates, variables, outcomes], axis=1)

In [33]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ FirstLineOnly', data=model_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                 3783
Model:                            GLM   Df Residuals:                     3781
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                         0.22043
Method:                          IRLS   Log-Likelihood:                -2506.5
Date:                Fri, 21 Jun 2019   Deviance:                       833.43
Time:                        22:47:01   Pearson chi2:                     833.
No. Iterations:                     3   Covariance Type:             nonrobust
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.3217      0.009     36.659      0.000       0.305       0.339
FirstLineOnly    -0.0128      0.009     -1.455      0.146      -0.030       0.004
=================================================================================
"""

In [34]:
np.exp(results.params[1:])

FirstLineOnly    0.987313
dtype: float64

In [35]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ FirstLineOnly + covariates', data=model_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                   59
Model:                            GLM   Df Residuals:                       45
Model Family:                Gaussian   Df Model:                           13
Link Function:               identity   Scale:                        0.082703
Method:                          IRLS   Log-Likelihood:                -2.1978
Date:                Fri, 21 Jun 2019   Deviance:                       3.7216
Time:                        22:47:03   Pearson chi2:                     3.72
No. Iterations:                     3   Covariance Type:             nonrobust
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5025      0.431      1.165      0.244      -0.343       1.348
FirstLineOnly     -0.1401      0.096     -1.456      0.145      -0.329       0.048
covariates[0]      0.0114      0.104      0.110      0.913      -0.192       0.215
covariates[1]     -0.0057      0.147     -0.039      0.969      -0.294       0.283
covariates[2]     -0.0543      0.097     -0.559      0.576      -0.245       0.136
covariates[3]      0.0743      0.099      0.751      0.452      -0.119       0.268
covariates[4]  -1.122e-16   2.87e-16     -0.391      0.696   -6.74e-16     4.5e-16
covariates[5]      0.1584      0.317      0.500      0.617      -0.462       0.779
covariates[6]      0.0807      0.131      0.614      0.539      -0.177       0.338
covariates[7]      0.0145      0.144      0.101      0.919      -0.267       0.297
covariates[8]      0.2227      0.180      1.234      0.217      -0.131       0.576
covariates[9]  -2.598e-17   3.14e-17     -0.827      0.408   -8.75e-17    3.56e-17
covariates[10]    -0.1857      0.133     -1.395      0.163      -0.447       0.075
covariates[11]     0.0622      0.055      1.130      0.258      -0.046       0.170
covariates[12]    -0.0332      0.045     -0.734      0.463      -0.122       0.055
covariates[13]    -0.0406      0.010     -4.074      0.000      -0.060      -0.021
==================================================================================
"""

In [36]:
np.exp(results.params[1:])

FirstLineOnly     0.869285
covariates[0]     1.011478
covariates[1]     0.994323
covariates[2]     0.947101
covariates[3]     1.077089
covariates[4]     1.000000
covariates[5]     1.171579
covariates[6]     1.084084
covariates[7]     1.014650
covariates[8]     1.249396
covariates[9]     1.000000
covariates[10]    0.830524
covariates[11]    1.064128
covariates[12]    0.967383
covariates[13]    0.960201
dtype: float64

In [37]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ OneFq', data=model_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                 3783
Model:                            GLM   Df Residuals:                     3781
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                         0.22022
Method:                          IRLS   Log-Likelihood:                -2504.8
Date:                Fri, 21 Jun 2019   Deviance:                       832.66
Time:                        22:47:03   Pearson chi2:                     833.
No. Iterations:                     3   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3517      0.013     27.969      0.000       0.327       0.376
OneFq          0.0297      0.013      2.366      0.018       0.005       0.054
==============================================================================
"""

In [38]:
np.exp(results.params[1:])

OneFq    1.030197
dtype: float64

In [39]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ OneInj', data=model_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                 3783
Model:                            GLM   Df Residuals:                     3781
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                         0.22013
Method:                          IRLS   Log-Likelihood:                -2504.0
Date:                Fri, 21 Jun 2019   Deviance:                       832.32
Time:                        22:47:04   Pearson chi2:                     832.
No. Iterations:                     3   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3139      0.009     33.804      0.000       0.296       0.332
OneInj        -0.0248      0.009     -2.674      0.007      -0.043      -0.007
==============================================================================
"""

In [40]:
np.exp(results.params[1:])

OneInj    0.975472
dtype: float64

In [41]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ OneFq + covariates', data=model_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                   59
Model:                            GLM   Df Residuals:                       45
Model Family:                Gaussian   Df Model:                           13
Link Function:               identity   Scale:                        0.086598
Method:                          IRLS   Log-Likelihood:                -3.5554
Date:                Fri, 21 Jun 2019   Deviance:                       3.8969
Time:                        22:47:05   Pearson chi2:                     3.90
No. Iterations:                     3   Covariance Type:             nonrobust
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5524      0.445      1.241      0.215      -0.320       1.425
OneFq              0.0026      0.104      0.025      0.980      -0.201       0.207
covariates[0]      0.0045      0.108      0.042      0.967      -0.208       0.217
covariates[1]     -0.0192      0.151     -0.127      0.899      -0.315       0.276
covariates[2]     -0.0404      0.104     -0.388      0.698      -0.244       0.164
covariates[3]      0.0398      0.101      0.395      0.693      -0.158       0.237
covariates[4]   1.951e-17   1.09e-16      0.179      0.858   -1.94e-16    2.33e-16
covariates[5]      0.1165      0.323      0.361      0.718      -0.517       0.750
covariates[6]      0.0774      0.135      0.575      0.565      -0.186       0.341
covariates[7]      0.0348      0.150      0.232      0.816      -0.258       0.328
covariates[8]      0.2403      0.185      1.298      0.194      -0.122       0.603
covariates[9]   5.042e-17   4.21e-17      1.197      0.231   -3.21e-17    1.33e-16
covariates[10]    -0.1536      0.136     -1.133      0.257      -0.419       0.112
covariates[11]     0.0783      0.055      1.413      0.158      -0.030       0.187
covariates[12]    -0.0413      0.046     -0.900      0.368      -0.131       0.049
covariates[13]    -0.0387      0.010     -3.805      0.000      -0.059      -0.019
==================================================================================
"""

In [42]:
np.exp(results.params[1:])

OneFq             1.002636
covariates[0]     1.004537
covariates[1]     0.981026
covariates[2]     0.960394
covariates[3]     1.040581
covariates[4]     1.000000
covariates[5]     1.123534
covariates[6]     1.080456
covariates[7]     1.035381
covariates[8]     1.271644
covariates[9]     1.000000
covariates[10]    0.857599
covariates[11]    1.081425
covariates[12]    0.959550
covariates[13]    0.962011
dtype: float64

In [43]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ OneInj + covariates', data=model_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                   59
Model:                            GLM   Df Residuals:                       45
Model Family:                Gaussian   Df Model:                           13
Link Function:               identity   Scale:                        0.086597
Method:                          IRLS   Log-Likelihood:                -3.5551
Date:                Fri, 21 Jun 2019   Deviance:                       3.8969
Time:                        22:47:06   Pearson chi2:                     3.90
No. Iterations:                     3   Covariance Type:             nonrobust
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5476      0.450      1.217      0.223      -0.334       1.429
OneInj             0.0020      0.064      0.032      0.975      -0.124       0.128
covariates[0]      0.0031      0.110      0.029      0.977      -0.212       0.218
covariates[1]     -0.0184      0.151     -0.122      0.903      -0.314       0.277
covariates[2]     -0.0420      0.102     -0.411      0.681      -0.242       0.158
covariates[3]      0.0400      0.101      0.395      0.693      -0.159       0.239
covariates[4]  -9.593e-18   2.95e-16     -0.032      0.974   -5.88e-16    5.69e-16
covariates[5]      0.1184      0.331      0.358      0.720      -0.530       0.767
covariates[6]      0.0772      0.134      0.574      0.566      -0.186       0.341
covariates[7]      0.0342      0.147      0.233      0.816      -0.253       0.322
covariates[8]      0.2406      0.186      1.297      0.195      -0.123       0.604
covariates[9]  -3.558e-17   1.19e-16     -0.299      0.765   -2.69e-16    1.98e-16
covariates[10]    -0.1540      0.137     -1.123      0.261      -0.423       0.115
covariates[11]     0.0792      0.060      1.317      0.188      -0.039       0.197
covariates[12]    -0.0419      0.050     -0.840      0.401      -0.140       0.056
covariates[13]    -0.0387      0.010     -3.786      0.000      -0.059      -0.019
==================================================================================
"""

In [44]:
np.exp(results.params[1:])

OneInj            1.002038
covariates[0]     1.003155
covariates[1]     0.981749
covariates[2]     0.958861
covariates[3]     1.040801
covariates[4]     1.000000
covariates[5]     1.125711
covariates[6]     1.080224
covariates[7]     1.034779
covariates[8]     1.271964
covariates[9]     1.000000
covariates[10]    0.857259
covariates[11]    1.082416
covariates[12]    0.958968
covariates[13]    0.962029
dtype: float64

In [45]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ FqOnly', data=model_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                 3783
Model:                            GLM   Df Residuals:                     3781
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                         0.22022
Method:                          IRLS   Log-Likelihood:                -2504.8
Date:                Fri, 21 Jun 2019   Deviance:                       832.66
Time:                        22:47:07   Pearson chi2:                     833.
No. Iterations:                     3   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3517      0.013     27.969      0.000       0.327       0.376
FqOnly         0.0297      0.013      2.366      0.018       0.005       0.054
==============================================================================
"""

In [46]:
np.exp(results.params[1:])

FqOnly    1.030197
dtype: float64

In [47]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ FqOnly + covariates', data=model_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                   59
Model:                            GLM   Df Residuals:                       45
Model Family:                Gaussian   Df Model:                           13
Link Function:               identity   Scale:                        0.086598
Method:                          IRLS   Log-Likelihood:                -3.5554
Date:                Fri, 21 Jun 2019   Deviance:                       3.8969
Time:                        22:47:08   Pearson chi2:                     3.90
No. Iterations:                     3   Covariance Type:             nonrobust
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5524      0.445      1.241      0.215      -0.320       1.425
FqOnly             0.0026      0.104      0.025      0.980      -0.201       0.207
covariates[0]      0.0045      0.108      0.042      0.967      -0.208       0.217
covariates[1]     -0.0192      0.151     -0.127      0.899      -0.315       0.276
covariates[2]     -0.0404      0.104     -0.388      0.698      -0.244       0.164
covariates[3]      0.0398      0.101      0.395      0.693      -0.158       0.237
covariates[4]   1.951e-17   1.09e-16      0.179      0.858   -1.94e-16    2.33e-16
covariates[5]      0.1165      0.323      0.361      0.718      -0.517       0.750
covariates[6]      0.0774      0.135      0.575      0.565      -0.186       0.341
covariates[7]      0.0348      0.150      0.232      0.816      -0.258       0.328
covariates[8]      0.2403      0.185      1.298      0.194      -0.122       0.603
covariates[9]   5.042e-17   4.21e-17      1.197      0.231   -3.21e-17    1.33e-16
covariates[10]    -0.1536      0.136     -1.133      0.257      -0.419       0.112
covariates[11]     0.0783      0.055      1.413      0.158      -0.030       0.187
covariates[12]    -0.0413      0.046     -0.900      0.368      -0.131       0.049
covariates[13]    -0.0387      0.010     -3.805      0.000      -0.059      -0.019
==================================================================================
"""

In [48]:
np.exp(results.params[1:])

FqOnly            1.002636
covariates[0]     1.004537
covariates[1]     0.981026
covariates[2]     0.960394
covariates[3]     1.040581
covariates[4]     1.000000
covariates[5]     1.123534
covariates[6]     1.080456
covariates[7]     1.035381
covariates[8]     1.271644
covariates[9]     1.000000
covariates[10]    0.857599
covariates[11]    1.081425
covariates[12]    0.959550
covariates[13]    0.962011
dtype: float64

In [49]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ InjOnly', data=model_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                 3783
Model:                            GLM   Df Residuals:                     3781
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                         0.22027
Method:                          IRLS   Log-Likelihood:                -2505.2
Date:                Fri, 21 Jun 2019   Deviance:                       832.86
Time:                        22:47:09   Pearson chi2:                     833.
No. Iterations:                     3   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3239      0.008     41.209      0.000       0.309       0.339
InjOnly       -0.0171      0.008     -2.170      0.030      -0.032      -0.002
==============================================================================
"""

In [50]:
np.exp(results.params[1:])

InjOnly    0.983086
dtype: float64

In [51]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ InjOnly + covariates', data=model_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                   59
Model:                            GLM   Df Residuals:                       45
Model Family:                Gaussian   Df Model:                           13
Link Function:               identity   Scale:                        0.086564
Method:                          IRLS   Log-Likelihood:                -3.5440
Date:                Fri, 21 Jun 2019   Deviance:                       3.8954
Time:                        22:47:13   Pearson chi2:                     3.90
No. Iterations:                     3   Covariance Type:             nonrobust
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5334      0.458      1.164      0.244      -0.365       1.431
InjOnly            0.0076      0.057      0.134      0.893      -0.103       0.119
covariates[0]      0.0041      0.106      0.038      0.969      -0.204       0.212
covariates[1]     -0.0112      0.161     -0.070      0.944      -0.326       0.304
covariates[2]     -0.0465      0.107     -0.436      0.663      -0.256       0.163
covariates[3]      0.0397      0.098      0.404      0.686      -0.153       0.232
covariates[4]   4.536e-17   1.32e-16      0.344      0.731   -2.13e-16    3.04e-16
covariates[5]      0.1237      0.328      0.378      0.706      -0.518       0.766
covariates[6]      0.0777      0.134      0.578      0.563      -0.186       0.341
covariates[7]      0.0376      0.149      0.252      0.801      -0.254       0.330
covariates[8]      0.2441      0.187      1.306      0.191      -0.122       0.610
covariates[9]    1.05e-16   9.31e-17      1.128      0.259   -7.75e-17    2.88e-16
covariates[10]    -0.1556      0.135     -1.148      0.251      -0.421       0.110
covariates[11]     0.0816      0.060      1.363      0.173      -0.036       0.199
covariates[12]    -0.0439      0.050     -0.880      0.379      -0.142       0.054
covariates[13]    -0.0385      0.010     -3.728      0.000      -0.059      -0.018
==================================================================================
"""

In [52]:
np.exp(results.params[1:])

InjOnly           1.007642
covariates[0]     1.004081
covariates[1]     0.988835
covariates[2]     0.954521
covariates[3]     1.040501
covariates[4]     1.000000
covariates[5]     1.131654
covariates[6]     1.080829
covariates[7]     1.038315
covariates[8]     1.276444
covariates[9]     1.000000
covariates[10]    0.855942
covariates[11]    1.084995
covariates[12]    0.957016
covariates[13]    0.962250
dtype: float64

In [53]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ FqInjectables', data=model_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                 3783
Model:                            GLM   Df Residuals:                     3781
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                         0.22016
Method:                          IRLS   Log-Likelihood:                -2504.3
Date:                Fri, 21 Jun 2019   Deviance:                       832.43
Time:                        22:47:16   Pearson chi2:                     832.
No. Iterations:                     3   Covariance Type:             nonrobust
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.3398      0.009     38.194      0.000       0.322       0.357
FqInjectables     0.0229      0.009      2.575      0.010       0.005       0.040
=================================================================================
"""

In [54]:
np.exp(results.params[1:])

FqInjectables    1.023173
dtype: float64

In [60]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ FqInjectables + covariates', data=model_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                   59
Model:                            GLM   Df Residuals:                       45
Model Family:                Gaussian   Df Model:                           13
Link Function:               identity   Scale:                        0.085872
Method:                          IRLS   Log-Likelihood:                -3.3069
Date:                Fri, 21 Jun 2019   Deviance:                       3.8642
Time:                        22:48:56   Pearson chi2:                     3.86
No. Iterations:                     3   Covariance Type:             nonrobust
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.6549      0.470      1.395      0.163      -0.265       1.575
FqInjectables      0.0319      0.052      0.617      0.537      -0.069       0.133
covariates[0]     -0.0009      0.106     -0.009      0.993      -0.209       0.207
covariates[1]     -0.0443      0.155     -0.285      0.775      -0.349       0.260
covariates[2]     -0.0316      0.100     -0.316      0.752      -0.227       0.164
covariates[3]      0.0379      0.098      0.388      0.698      -0.154       0.229
covariates[4]   2.181e-16   3.11e-16      0.700      0.484   -3.92e-16    8.28e-16
covariates[5]      0.0890      0.324      0.275      0.784      -0.547       0.725
covariates[6]      0.0746      0.134      0.557      0.578      -0.188       0.337
covariates[7]      0.0055      0.153      0.036      0.972      -0.295       0.306
covariates[8]      0.2129      0.189      1.129      0.259      -0.157       0.583
covariates[9]   1.086e-16   5.69e-17      1.910      0.056   -2.86e-18     2.2e-16
covariates[10]    -0.1447      0.134     -1.076      0.282      -0.408       0.119
covariates[11]     0.0634      0.060      1.055      0.291      -0.054       0.181
covariates[12]    -0.0281      0.050     -0.557      0.577      -0.127       0.071
covariates[13]    -0.0407      0.011     -3.858      0.000      -0.061      -0.020
==================================================================================
"""

In [61]:
np.exp(results.params[1:])

FqInjectables     1.032452
covariates[0]     0.999076
covariates[1]     0.956660
covariates[2]     0.968879
covariates[3]     1.038617
covariates[4]     1.000000
covariates[5]     1.093100
covariates[6]     1.077419
covariates[7]     1.005466
covariates[8]     1.237241
covariates[9]     1.000000
covariates[10]    0.865283
covariates[11]    1.065422
covariates[12]    0.972297
covariates[13]    0.960132
dtype: float64

In [57]:
import statsmodels.formula.api as smf
model = smf.glm('Outcome ~ OneFq + OneInj + FqOnly + InjOnly + FqInjectables + covariates', data=model_data)
results = model.fit()
results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Outcome   No. Observations:                   59
Model:                            GLM   Df Residuals:                       42
Model Family:                Gaussian   Df Model:                           16
Link Function:               identity   Scale:                        0.088458
Method:                          IRLS   Log-Likelihood:                -2.1470
Date:                Fri, 21 Jun 2019   Deviance:                       3.7152
Time:                        22:47:20   Pearson chi2:                     3.72
No. Iterations:                     3   Covariance Type:             nonrobust
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.8210      0.508      1.616      0.106      -0.175       1.817
OneFq              0.0675      0.070      0.964      0.335      -0.070       0.205
OneInj            -0.0019      0.086     -0.022      0.982      -0.170       0.166
FqOnly             0.0675      0.070      0.964      0.335      -0.070       0.205
InjOnly            0.1339      0.115      1.168      0.243      -0.091       0.359
FqInjectables      0.1477      0.104      1.425      0.154      -0.056       0.351
covariates[0]      0.0100      0.115      0.086      0.931      -0.216       0.236
covariates[1]     -0.0178      0.166     -0.107      0.914      -0.342       0.307
covariates[2]     -0.0486      0.112     -0.435      0.664      -0.268       0.170
covariates[3]      0.0716      0.109      0.658      0.511      -0.142       0.285
covariates[4]  -5.152e-18   1.02e-16     -0.050      0.960   -2.06e-16    1.96e-16
covariates[5]      0.1428      0.336      0.425      0.671      -0.515       0.800
covariates[6]      0.0796      0.136      0.584      0.559      -0.187       0.347
covariates[7]      0.0032      0.157      0.020      0.984      -0.305       0.311
covariates[8]      0.2113      0.191      1.104      0.270      -0.164       0.586
covariates[9]   9.706e-18   4.49e-17      0.216      0.829   -7.82e-17    9.76e-17
covariates[10]    -0.1800      0.141     -1.275      0.202      -0.457       0.097
covariates[11]     0.0556      0.064      0.866      0.387      -0.070       0.181
covariates[12]    -0.0272      0.053     -0.519      0.604      -0.130       0.076
covariates[13]    -0.0414      0.011     -3.862      0.000      -0.062      -0.020
==================================================================================
"""

In [59]:
np.exp(results.params[1:])

OneFq             1.069793
OneInj            0.998078
FqOnly            1.069793
InjOnly           1.143329
FqInjectables     1.159213
covariates[0]     1.010011
covariates[1]     0.982366
covariates[2]     0.952585
covariates[3]     1.074254
covariates[4]     1.000000
covariates[5]     1.153444
covariates[6]     1.082802
covariates[7]     1.003182
covariates[8]     1.235241
covariates[9]     1.000000
covariates[10]    0.835290
covariates[11]    1.057157
covariates[12]    0.973128
covariates[13]    0.959447
dtype: float64